In [5]:
import bz2

# 원본 압축 파일과 해제할 파일 이름 설정
bz2_file = "C:/face/shape_predictor_68_face_landmarks.dat.bz2"
output_file = "C:/face/shape_predictor_68_face_landmarks.dat"

# 압축 해제
with bz2.BZ2File(bz2_file, "rb") as fr, open(output_file, "wb") as fw:
    fw.write(fr.read())

print("압축 해제 완료!")


압축 해제 완료!


In [6]:
import bz2

# 원본 압축 파일과 해제할 파일 이름 설정
bz2_file = "C:/face/dlib_face_recognition_resnet_model_v1.dat.bz2"
output_file = "C:/face/dlib_face_recognition_resnet_model_v1.dat"

# 압축 해제
with bz2.BZ2File(bz2_file, "rb") as fr, open(output_file, "wb") as fw:
    fw.write(fr.read())

print("압축 해제 완료!")


압축 해제 완료!


In [ ]:
import dlib
import cv2
import numpy as np
import os

# 얼굴 감지기 및 얼굴 인코더 모델 로드
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/face/shape_predictor_68_face_landmarks.dat")
face_rec_model = dlib.face_recognition_model_v1("C:/face/dlib_face_recognition_resnet_model_v1.dat")

# myface 폴더에서 기준 얼굴 이미지 로드
face_encodings = []
face_names = []
myface_dir = "C:/myface"

for file in os.listdir(myface_dir):
    img_path = os.path.join(myface_dir, file)
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = detector(img_rgb)
    
    if len(faces) > 0:
        shape = predictor(img_rgb, faces[0])
        face_descriptor = face_rec_model.compute_face_descriptor(img_rgb, shape)
        face_encodings.append(np.array(face_descriptor))
        face_names.append(file.split('.')[0])  # 파일명을 이름으로 저장
    
print(f"Loaded {len(face_encodings)} face encodings from {myface_dir}")

# 실시간 웹캠 얼굴 인식
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector(img_rgb)
    
    for face in faces:
        shape = predictor(img_rgb, face)
        face_descriptor = face_rec_model.compute_face_descriptor(img_rgb, shape)
        face_encoding = np.array(face_descriptor)
        
        # 유사도 비교
        min_dist = float("inf")
        identity = "Unknown"
        
        for stored_encoding, name in zip(face_encodings, face_names):
            dist = np.linalg.norm(stored_encoding - face_encoding)
            if dist < min_dist:
                min_dist = dist
                identity = name
                
        # 유사도 출력
        similarity = 1 - (min_dist / 1.0)  # 정규화 (거리가 0이면 1, 클수록 0)
        
        # 얼굴 박스 그리기
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = f"{identity}: {similarity:.2f}"
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    cv2.imshow("Face Recognition", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
